#### Image Classification
- Challenges
    - Viewpoint variation : 시점 변화
    - Scale Variation : 크기 변화
    - Deformation : 변형
    - Occlusion : 가림
    - Illumination conditions : 조명 조건
    - Background clutter : 배경 혼란
    - Intra-class variation : 클래스 내  다양성 ‘의자(chair)’처럼 하나의 클래스 안에도 다양한 형태와 외형을 가진 객체들이 존재할 수 있습니다

#### Nearest Neighbor Classifier
- Simply use L1 distance 
- L1 distance : ∑ |I1ᵢ - I2ᵢ|
- L2 distance : `distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis=1))`
- 사실 실제 코드에서는 굳이 np.sqrt를 할 필요는 없음. 하나 안하나 순서는 동일하기 때문에..
- L1 vs L2
    - L1 : 전체적으로 픽셀 차이를 고르게 더함. 이상치에 덜 민감
    - L2 : 이상치에 매우 민감. 작은 일부 픽셀의 큰 차이가 전체 거리에 큰 영향을 줌
    - 정밀한 차이를 잘 반영하고 싶을 때는 L2 distance, 이상치가 많을 때는 L1 distance


In [2]:
from cs231n.data_utils import load_CIFAR10

Xtr, Ytr, Xte, Yte = load_CIFAR10('data/cifar10/')

In [5]:
Xtr.shape, Ytr.shape, Xte.shape, Yte.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

In [6]:
# 1dimensional로 flatten

width = Xtr.shape[1]
height = Xtr.shape[2]
im_channel = Xtr.shape[3]
Xtr_rows = Xtr.reshape(Xtr.shape[0], width*height*im_channel)
Xte_rows = Xte.reshape(Xte.shape[0], width*height*im_channel)

In [17]:
import numpy as np
from collections import Counter

class NearestNeighbor():
    def __init__(self):
        pass

    def train(self, X, y):
        # NearestNeighbor는 단순히 X, y를 복사
        self.Xtr = X
        self.ytr = y

    def predict(self, X, k=1, method='L1'):
        num_test = X.shape[0]
        Ypred = np.zeros(X.shape[0], dtype=self.ytr.dtype)

        for i in range(num_test):
            if method == 'L1':
                distances = np.sum(np.abs(self.Xtr - X[i, :]), axis=1)
            elif method == 'L2':
                distance = np.sqrt(np.sum(np.square(self.Xtr - X[i, :]), axis=1))
            
            # 단순 NearestNeighbor 모델일때
            # min_index = np.argmin(distances)
            # Ypred[i] = self.ytr[min_index]

            # 밑에는 k-NN
            # 가까운 k개 인덱스
            nearest_idxs = np.argsort(distances)[:k]
            nearest_labels = self.ytr[nearest_idxs]

            # 다수결 투표 (가장 많이 나온 라벨 선택)
            most_common = Counter(nearest_labels).most_common(1)[0][0]
            Ypred[i] = most_common
        
        return Ypred

In [22]:
Counter(np.array([1,2,1,1,1,1,])).most_common(1)

[(np.int64(1), 5)]

In [13]:
nn = NearestNeighbor()
nn.train(Xtr_rows, Ytr)

In [16]:
y_pred = nn.predict(Xte_rows)
accuracy = np.mean(y_pred == Yte)

#### k-Nearest Neighbor Classifier
- 학습 데이터에서 가장 가까운 이미지 하나만 찾는 것이 아니라 가장 가까운 k개의 이미지를 찾고
- 이 k개가 투표해서 테스트 이미지의 라벨을 결정
- k=1인 경우 기존의 Nearest Neighbor Classifier와 동일
- k가 클수록 결과가 smoothing, 이상치에 강하며 noise에 덜 민감한 분류기를 만들 수 있다.
- 코드는 위에!
- 이상적인 k는 테스트를 해봐야한다. --> cross-validation

#### Validation sets for Hyperparameter tuning
- k값을 설정할 때, 여러 값을 시도해보고 가장 잘 되는 걸 고르면?
- 단, 하이퍼파라미터 튜닝을 위해서 테스트셋을 사용하는건 안됨
    - overfitting
    - 실제 서비스 시, 성능이 확 떨어질 가능성이 있다.
    - 테스트셋을 학습 데이터처럼 쓴 것이고 성능이 과하게 낙관적으로 나올 수 있다.

- 올바른 validation set 사용법
    - 학습데이터를 두 개로 나눈다. (train/validation)

In [ ]:
# assume we have Xtr_rows, Ytr, Xte_rows, Yte as before
# recall Xtr_rows is 50,000 x 3072 matrix
Xval_rows = Xtr_rows[:1000, :] # take first 1000 for validation
Yval = Ytr[:1000]
Xtr_rows = Xtr_rows[1000:, :] # keep last 49,000 for train
Ytr = Ytr[1000:]

# find hyperparameters that work best on the validation set
validation_accuracies = []
for k in [1, 3, 5, 10, 20, 50, 100]:
    # use a particular value of k and evaluation on validation data
    nn = NearestNeighbor()
    nn.train(Xtr_rows, Ytr)
    # here we assume a modified NearestNeighbor class that can take a k as input
    Yval_predict = nn.predict(Xval_rows, k = k)
    acc = np.mean(Yval_predict == Yval)
    #   print 'accuracy: %f' % (acc,)

    # keep track of what works on the validation set
    validation_accuracies.append((k, acc))

#### Cross-validation
- 학습데이터의 크기가 작을 경우 또는 validation set이 분할 방식에 따라 편향되거나 하는 경우가 있을 수 있다. 
- 여러 가지 validation set을 번갈아 사용하고, 그 결과를 평균 내는 방식
- 5-fold cross validation
    - 학습 데이터를 5개의 동일한 크기로 나누고
    - 그 중 4개는 학습용, 1개는 검증용으로 사용한다.
    - 검증용 폴드 위치를 바꿔가며 5번 반복
    - 각 반복에서의 성능을 기록하고 최종적으로 평균을 내어 k의 성능을 평가한다.

- 장점
    - 데이터가 작아도 검증 결과의 신뢰도가 높아진다.
    - 특정 데이터 조합에 과적합 되는 것을 방지
    - 모델이 일관적으로 잘 작동하는지 확인이 가능하다.
- 단점
    - computationally expensive
    